<a href="https://colab.research.google.com/github/siddharthabhuyan86/geowebapp/blob/main/tq_17dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Hands-on Introduction to Quantum Kernel Methods:[TorchQuantum](https://torchquantum.readthedocs.io/en/main/index.html)
###1. TorchQuantum Basics

*  Single-qubit gates
*  Superposition
*  Measurement
* Two-qubit gates and entanglement state


###**Set up**

In [ ]:
print('Installing torchquantum...')
!git clone https://github.com/mit-han-lab/torchquantum.git
%cd /content/torchquantum
!pip install --editable .

In [ ]:
import torchquantum as tq
import torchquantum.functional as tqf
import numpy as np
import matplotlib.pyplot as plt
import torch
import numpy

In [ ]:
# Qubit
n_qubits=1
q_device=tq.QuantumDevice(n_wires=n_qubits)
print(q_device)

In [ ]:
#print the states


###Gates
[Bloch sphere](https://javafxpert.github.io/grok-bloch/)




In [ ]:
# X gate
q_device.reset_states(bsz=1)
tqf.x(q_device, wires=0)
states=q_device.get_states_1d()[0]
print(f"after x gate: {states}")

In [ ]:
#Hadamard gate


###what the final state should be for the circuits below
1. XX|0>
2. HZH|1>

###Measurement

In [ ]:
#Hadamard gate
q_device.reset_states(bsz=1)
tqf.h(q_device, wires=0)
states=q_device.get_states_1d()[0]
print(f"after h gate: {states}")

bitstring = tq.measure(q_device, n_shots=1024, draw_id=0)
print(bitstring)

In [ ]:
bitstring = tq.measure(q_device, n_shots=1024, draw_id=0)
print(bitstring)

#Entangled state


In [ ]:
q_device = tq.QuantumDevice(n_wires=2)
q_device.reset_states(bsz=1)
print("Initial quantum state:", q_device.get_states_1d()[0])

In [ ]:
# RY gate
q_device.reset_states(bsz=1)
tqf.ry(q_device, wires=0, params=[np.pi/2])
states=q_device.get_states_1d()[0]
print(f"after ry gate: {states}")

In [ ]:
# Simulate encoding a data point
data_point = np.array([0,np.pi/4,np.pi/2])
print(f"Goal: Encode classical data point x = {data_point} into quantum state")

In [ ]:
q_device = tq.QuantumDevice(n_wires=3)
q_device.reset_states(bsz=1)
print("Initial quantum state:", q_device.get_states_1d()[0])
# Angle encoding: map data to RY rotations

for i in range(len(data_point)):
    tqf.ry(q_device, wires=i, params=[data_point[i]])
states = q_device.get_states_1d()[0]
print("Encoded quantum state:", states)

In [ ]:
#print probabilities

# 2. Quantum Kernel Methods for moons dataset classification

###Outline
1. Introduction to Quantum Kernel Methods.
2. Build and train an SVM using Quantum Kernel Methods.


In [ ]:
from sklearn.datasets import make_moons,make_circles
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#preprocessiong
X, y = make_circles(n_samples=200, noise=0.1, factor=0.5, random_state=42)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler(feature_range=(0, np.pi))
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)
print(f"Training samples: {len(X_train)}")
print(f" Test samples: {len(X_test)}")
print(X_test)
print(y_test)
# Visualize the dataset
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train,
                     cmap='RdBu', s=100, edgecolors='black', linewidth=1.5)
plt.colorbar(scatter, label='Class')
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('Make Moons Dataset\n(Non-linearly Separable)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
class KernelAnsatz(tq.QuantumModule):
    def __init__(self):
        super().__init__()
    def forward(self, q_device: tq.QuantumDevice, x, y):
        q_device.reset_states(bsz=x.shape[0])

        # Step 2: Encode x
        tqf.ry(q_device, wires=0, params=x[:, 0])
        tqf.ry(q_device, wires=1, params=x[:, 1])


        # Step 3: Apply inverse encoding of y
        tqf.ry(q_device, wires=1, params=-y[:, 1])
        tqf.ry(q_device, wires=0, params=-y[:, 0])

In [ ]:
class Kernel(tq.QuantumModule):
    def __init__(self):
        super().__init__()
        self.n_wires = 2
        self.q_device = tq.QuantumDevice(n_wires=self.n_wires)
        self.ansatz = KernelAnsatz()

    def forward(self, x, y):
        # Ensure batch format
        x = x.view(1, -1)
        y = y.view(1, -1)

        # Apply quantum circuit
        self.ansatz(self.q_device, x, y)

        kernel_value = torch.abs(
            self.q_device.get_states_1d()[0, 0]
        )
        return kernel_value

In [ ]:
kernel_function = Kernel()
def kernel_matrix(A, B):
    return np.array([[kernel_function(a, b) for b in B] for a in A])

svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)
predictions = svm.predict(X_test)
print(accuracy_score(predictions, y_test))